# Selectividad Exams Parser

## Libraries import

### Required packages for OCR processing:
**Ubuntu based:**
- tesseract-ocr
- tesseract-ocr-spa
- poppler-utils

**Archlinux based:**
- tesseract
- tesseract-data-spa

In [1]:
# !pip install pdf2image
# !pip install pytesseract

In [2]:
from pdf2image import convert_from_path
import pytesseract
import pandas as pd
from pathlib import Path
from multiprocessing import Pool
import os
import requests
from datetime import datetime

## Functions

### Telegram bot

Function to generate messages to a telegram bot

In [3]:
# Replace with your bot's token and chat ID
from bot_credentials import *

def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    params = {"chat_id": CHAT_ID, "text": message}
    response = requests.get(url, params=params)
    return response.json()

### *pdf_to_text*

Extract the text from pdf files, page by page

In [4]:
def pdf_to_text(file_path):
    """
    Processes a single PDF file to extract exam information and statements.

    Args:
        file (Path): A Path object representing the PDF file to be processed.
    
    Returns:
        dict: A dictionary containing the extracted information with the following structure:
            {
                "key": {
                    "subject": list of subjects,
                    "year": list of years,
                    "topic": list of topics,
                    "exam": list of exams,
                    "exercise": list of exercises,
                    "statement": list of statements
                }
            }

    Workflow:
        1. Extracts the topic from the file name.
        2. Converts the PDF file into text, page by page.
        3. For each page:
            - Extracts the statement using the `extract_statement` function.
            - Extracts exam details using the `extract_exam_details` function.
            - Handles errors by logging them using `generate_error_log`.
        4. Aggregates the extracted information into lists for subjects, years, topics, exams, exercises, and statements.
        5. Returns a dictionary with the extracted data.

    Notes:
        - Logs errors to a file if there are issues with extracting statements or exam details.
        - Skips pages with errors and continues processing the remaining pages.
    """
    images = convert_from_path(file_path)[1:]
    # parse each page
    pages = []
    for i, page in enumerate(images):
        pages.append(pytesseract.image_to_string(page, lang='spa'))
    return pages

### *extract_statement*
Extract the statement from a page in text format

In [5]:
def extract_statement(page):
    # split the content into lines
    text = page.split('\n')

    # list to store the processed lines
    final_text = []
    # start from the 4th line
    for line in text[4:]:

        # check the first word of the line
        first_word = line.replace('.', '').split(' ')[0].strip().upper()

        # stop at examen details line
        if first_word in ['SOCIALES', 'FISICA', 'MATES',
                          'QUÍMICA', 'QUIMICA', 'QUIÍMICA']:
            final_text.append(line.strip())
            break
        
        # add the line to final_text
        else:
            if not line == '':
                final_text.append(line.strip())

    # join the list into a single string
    final_text = '\n'.join(final_text)

    return final_text

In [6]:
'a' in ['a', 'b']

True

### *extract_exam_details*
Extract the exam details, like subject, year, exam and exercise number.

In [7]:
def extract_exam_details(statement):
    # get last line with exam details
    lines = [i for i in statement.split('\n') if i != '']
    exam_details = lines[-1].lower()

    # strip whitespaces
    exam_details = (exam_details.replace('.', '')            
                    .replace(',', '')
                    .replace('(', '')
                    .replace(')', '').strip()
    )
    
    # split the words by ' '
    exam_details = exam_details.split(' ')

    # MATES CCSS
    # SOCIALES II. 2017 JUNIO. EJERCICIO 2. OPCIÓN A
    # 
    if exam_details[0].startswith('sociales'):
        exam_details[0] = 'MATES_CCSS'
        # remove the second element in the list
        del exam_details[1]
    
       
    # exam dictionary
    details = ['subject', 'year', 'exam', 'exercise']
    exam_dict = dict.fromkeys(details)
    exam_dict = {k:None for k in details}

    ## fill dictionary with values
    
    # subject
    exam_dict['subject'] = exam_details[0]
    
    # year
    # SOCIALES II. PONENCIA 2009. EJERCICIO 2
    if exam_details[1] == 'ponencia':
        # switch ponencia and year
        exam_details[1], exam_details[2] = exam_details[2], exam_details[1]

    exam_dict['year'] = int(exam_details[1])

    # parse the exam string
    if exam_details[2].startswith('reserva'):
        exam_dict['exam'] = ' '.join(exam_details[2:4]).title()
        exam_index_start = 4
    else:
        exam_dict['exam'] = exam_details[2].title()
        exam_index_start = 3
    
    exam_dict['exercise'] = ' '.join(exam_details[exam_index_start:]).title()

    
    return exam_dict

### *time*
Function to measure the time taken to process the files

In [8]:
import time
def what_time():
    now = time.localtime()
    time_formated = time.strftime("%H:%M:%S on %Y-%m-%d ", now)
    return now, time_formated

def elapsed_time_minutes(start, end):
    start_seconds = time.mktime(start)
    end_seconds = time.mktime(end)
    return (end_seconds - start_seconds) / 60

### *generate_error_log*
generate a error log and add it to errors_timestamp.log file

In [9]:
def generate_error_log(file, page, error_type, statement = None, details = None):
    log = f'Error processing {error_type}, in {file.name} at page {page+2}\n'
    
    # add timestamp to log file
    current_datetime = datetime.now()
    formatted_dt = current_datetime.strftime("%d-%m-%Y")
    with open(f'./error_logs/errors_{formatted_dt}.log', 'a') as f:
        f.write(log)
        f.write('*' * 10 + '\n')
        if statement:
            f.write(f'{statement}\n')
        elif details:
            f.write(f'{details}\n')
        f.write('*' * 10 + '\n')
    
    # print log to console without the new line
    print(log[:-2])

### *add_row*
Manually add a row to the processed exercises

In [10]:
def add_row(df, statement, exam_details):
    
    row = extract_exam_details(exam_details)
    row['statement'] = statement

    return pd.concat([df, pd.DataFrame(row, columns = df.columns, index = [0])], axis = 0)

### *process_file*
Process a pdf files, extracting its information: statement and exam information 

In [ ]:
def process_file(file: str) -> dict:
    exercises_dict = {}
    file = Path(file)
    topic = file.stem.split(' - ')[-1]   
    
    # lists to store the values
    subjects = []
    years = []
    exams = []
    exercises = []
    statements = []
    pages = pdf_to_text(file)

    # parse each page in the pdf file

    output = f'Success parsing file: {file.stem}'
    
    for index, page in enumerate(pages):

        # extract statement from page
        try:
            statement = extract_statement(page)
        # error with extracting statement
        except:
            for line in statement.split('\n'):
                if line.startswith('www.emestrada.org'):
                    continue
            generate_error_log(file = file, page = index, error_type= 'statement', statement=page)
            continue

        # extract exam details
        try:
            details = extract_exam_details(statement)
            subjects.append(details['subject'])
            years.append(details['year'])
            exams.append(details['exam'])
            exercises.append(details['exercise'])
            statements.append(statement)
        # error with extracting details
        except:
            # pages ending with www.emestrada.org are pages with solution to exercise
            if not statement.endswith('www.emestrada.org'):
                generate_error_log(file = file, page = index, error_type= 'details', details = statement)
                output = f'File {file.stem} parsed but with errors'
            continue
        
    print(output)

    # generate the key and content for the exercises dictionary
    if details:
        key = details['subject'] + ' ' + str(details['year']) + ' ' + topic
        exercises_dict[key] = {
            'subject' : subjects,
            'year' : years,
            'topic' : [topic] * len(subjects),
            'exam' : exams,
            'exercise' : exercises,
            'statement' : statements
        }
        return exercises_dict

### *create_content_dict*
Creates a python dictionary with the exam details and its statement

In [12]:
def create_content_dict():
    # create the dict from keys
    keys = ['subject', 'year', 'topic', 'exam', 'exercise', 'statement']
    #content = dict.fromkeys(exercises_dict[first_processed_file].keys())
    content = dict.fromkeys(keys)
    # fill the dict with empty lists
    for key in content.keys():
        content[key] = []

    return content

### *process_folder*
Processes all files within a folder

In [13]:
def process_folder(folder:str, csv_file:str = None, return_df: bool = False) -> pd.DataFrame:
    df = pd.DataFrame()
    
    # get the list of pdf files within the folder
    folder_Path = Path(folder)
    files = sorted( [i for i in Path.iterdir(folder_Path)
                        if i.suffix == '.pdf'] )
    
    # process each pdf file to extract its information
    for file in files:
        try:
            exercises_dict = process_file(file)
            # each year contains a dictionary with keys containing lists of values
            for year, dict_ in exercises_dict.items():
                df = pd.concat([df, pd.DataFrame(dict_)], axis = 0)

        except:
            pass
        
    # correct wrong subjects
    #df.subject = df.subject.apply(lambda x: 'Química'
    #                                if x.endswith('mica') else x)
    
    # export the output of the processed pdf files to a csv file
    if csv_file:  # if a file name is provided
        output_file = f'./csv/{csv_file}.csv'
    else:
        output_file = f'./csv/exercises_{folder_Path.stem}.csv'
    
    df.to_csv(output_file, index=False)

    if return_df:
        return df

## Process pdf files

In [14]:
folders = sorted([i for i in Path('./pdf_files/').iterdir() if i.is_dir()])
folders = folders[:-1]
folders

[PosixPath('pdf_files/FISICA'),
 PosixPath('pdf_files/MATES_CCSS'),
 PosixPath('pdf_files/QUIMICA')]

In [ ]:
def process_all_files_in_folder(folder:str) -> None:
    start, start_formatted = what_time()
    send_telegram_message(f'Parsing process started at: {start_formatted}')

    subject_df = pd.DataFrame()
    folder = Path(folder)
    subfolders = sorted(folder.iterdir())
    for subfolder in subfolders:
        try:
            
            topic_df = process_folder(subfolder, return_df = True)
            print(f'Success processing folder: {subfolder.stem}')
            send_telegram_message(f'Success processing folder: {folder.stem}')
            subject_df = pd.concat([subject_df, topic_df], axis = 0)
            
        except Exception as e:
            send_telegram_message(f'Error processing folder: {subfolder.stem}' \
                                f'\n{e}')
    
    return subject_df

    end, end_formatted = what_time()

    send_telegram_message(f'Parsing process finished at: {end_formatted}'\
                        f'\nTime elapsed in minutes: {elapsed_time_minutes(start, end)}')
    send_telegram_message('✅ Your files have been processed!')

In [ ]:
# QUIMICA_df = process_all_files_in_folder('./pdf_files/QUIMICA')

In [ ]:
# for col in QUIMICA_df.columns[:-1]:
#     print(f'{col}: {QUIMICA_df[col].unique()}')

In [ ]:
# df.to_csv('./csv/QUIMICA.csv', index=False)

## Testing

### Get random files to test parsing

In [ ]:
from numpy.random import choice, seed

In [ ]:
def sample_topics(files_per_folder:int = 5) -> list:
    sample_files = []
    seed(42)
    subjects = sorted([i for i in Path('./pdf_files/').iterdir() if i.is_dir()])
    # remove samples folder
    subjects = subjects[:-1]

    sample_files = []
    for s in subjects:
        for subfolder in s.iterdir():
            print(f'Processing {subfolder.stem}')
            files = list(subfolder.glob('**/*.pdf'))
            sample_files.extend(choice(files, files_per_folder, replace = False))  

    print(f'{len(sample_files)} files sampled')

    return sample_files 
    

In [ ]:
def sample_subjects(files_per_folder:int = 3) -> list:
    sample_files = []
    seed(42)

    for f in folders:
        
        files = list(f.glob("**/*.pdf"))
        sample_files.extend(choice(files,files_per_folder, replace=False))

    print(f'Files selected: {len(sample_files)}')
    
    return sorted(sample_files)    

In [ ]:
sample_files = sample_subjects(5)

In [ ]:
sample_files = sample_topics(2)

### Save the sample files into folder

In [ ]:
import shutil

In [ ]:
os.makedirs('pdf_files/sample_files', exist_ok=True)

In [ ]:
for file in sample_files:
    shutil.copy(file, f'pdf_files/sample_files/')

### Process the sample files 

In [ ]:
df = process_folder('pdf_files/sample_files')

In [ ]:
df

In [ ]:
for col in df.columns[:-1]:
    print(f'{col}: {df[col].unique()}')

In [ ]:
pages = pdf_to_text('pdf_files/folder/2003 - Conf. Electrónica.pdf')

In [ ]:
st = extract_statement(pages[0])
print(st)

In [ ]:
extract_exam_details(st)

In [ ]:
df = pd.read_csv('csv/exercises_sample_files.csv')

## Check processing of files

1. Create path to file to process
2. Use `pdf_to_text` to process the pdf file. Store output in `pages`
3. Extract statement using `extract_statement`

In [ ]:
file = Path('pdf_files/MATES_CCSS/Contraste de Hipótesis/2016 - Contraste De Hipótesis.pdf')
pages = pdf_to_text(file)

In [ ]:
statement = extract_statement(pages[0])
statement.split('\n')[-1]

In [ ]:
details = extract_exam_details(statement)
details

In [ ]:
statement.split('\n')[-1]

## Manual Addition of exercises

#### Load csv into a dataframe

In [59]:
df = pd.read_csv('./csv/QUIMICA/exercises_Solubilidad.csv')

In [62]:
file = Path('pdf_files/QUIMICA/Solubilidad/2023 - Solubilidad.pdf')
pages = pdf_to_text(file)

In [66]:
st = extract_statement(pages[3])
st

'El pH de una disolución acuosa saturada de Pb(OH), es 99 a 25"C. Basándose en la reacción\nquímica correspondiente, calcule:'

In [61]:
QUIMICA_df

,subject,year,topic,exam,exercise,statement
0,Química,2012,Equilibrio Químico,Junio,Ejercicio 5 Opción B,"PCl;(g) Z2 PCl,(g) + Cl,(g)\nCuando se alcanza..."
1,Química,2012,Equilibrio Químico,Reserva 1,Ejercicio 6 Opción A,a) El valor de Kc.\nb) El grado de disociación...
2,Química,2012,Equilibrio Químico,Reserva 2,Ejercicio 3 Opción A,a) Aumentar la temperatura.\nb) Retirar del re...
3,Química,2012,Equilibrio Químico,Reserva 2,Ejercicio 6 Opción B,temperatura de 11 *C la presión es de 0?3 atm....
4,Química,2012,Equilibrio Químico,Reserva 3,Ejercicio 3 Opción A,"reacción en cada uno de los siguientes casos, ..."
...,...,...,...,...,...,...
2,Química,2023,Enlace Químico,Reserva 2,Ejercicio B3,"Los átomos A, B, C y D corresponden a elemento..."
3,Química,2023,Enlace Químico,Reserva 3,Ejercicio B3,Justifique si las siguientes afirmaciones son ...
4,Química,2023,Enlace Químico,Reserva 4,Ejercicio B3,Justifique si las siguientes sustancias son co...
5,Química,2023,Enlace Químico,Julio,Ejercicio B3,Responda a las siguientes cuestiones de manera...


In [67]:
statement = '''El pH de una disolución acuosa saturada de Pb(OH)2 es 9’9 a 25ºC. Basándose en la reacción
química correspondiente, calcule:
a) La solubilidad molar en agua y el producto de solubilidad del Pb(OH) a 25ºC.
b) La solubilidad del Pb(OH)2 en una disolución de NaOH 0’1 M.'''


# SOCIALES II. 2017 JUNIO. EJERCICIO 2. OPCIÓN A
exam_details = 'QUÍMICA. 2023. RESERVA 2. EJERCICIO C2'

In [68]:
extract_exam_details(exam_details)

{'subject': 'química',
 'year': 2023,
 'exam': 'Reserva 2',
 'exercise': 'Ejercicio C2'}

In [69]:
df = add_row(QUIMICA_df, statement, exam_details)

In [73]:
df

,subject,year,topic,exam,exercise,statement
0,Química,2012,Equilibrio Químico,Junio,Ejercicio 5 Opción B,"PCl;(g) Z2 PCl,(g) + Cl,(g)\nCuando se alcanza..."
1,Química,2012,Equilibrio Químico,Reserva 1,Ejercicio 6 Opción A,a) El valor de Kc.\nb) El grado de disociación...
2,Química,2012,Equilibrio Químico,Reserva 2,Ejercicio 3 Opción A,a) Aumentar la temperatura.\nb) Retirar del re...
3,Química,2012,Equilibrio Químico,Reserva 2,Ejercicio 6 Opción B,temperatura de 11 *C la presión es de 0?3 atm....
4,Química,2012,Equilibrio Químico,Reserva 3,Ejercicio 3 Opción A,"reacción en cada uno de los siguientes casos, ..."
...,...,...,...,...,...,...
3,Química,2023,Enlace Químico,Reserva 3,Ejercicio B3,Justifique si las siguientes afirmaciones son ...
4,Química,2023,Enlace Químico,Reserva 4,Ejercicio B3,Justifique si las siguientes sustancias son co...
5,Química,2023,Enlace Químico,Julio,Ejercicio B3,Responda a las siguientes cuestiones de manera...
0,Química,2024,Enlace Químico,Junio,Ejercicio B3,Dados tres elementos cuyas configuraciones ele...


In [72]:
df.topic.fillna('Solubilidad', inplace=True)

In [39]:
df.topic.fillna('Matrices y Determinantes', inplace=True)

In [41]:
df.to_csv(f'/home/daniel/git_code/emestrada/csv/MATES CCSS/MATES_CCSS.csv', index=False)

## Crete combined csv file for subject

Create the pandas dataframe

In [ ]:
csv_files = [i for i in Path.iterdir(Path('./csv'))
             if i.suffix == '.csv' and not i.stem.startswith('all')]

In [ ]:
csv_files

In [ ]:
df = pd.read_csv(csv_files[0])

In [ ]:
# df = pd.read_csv(csv_files[0])
# for file in csv_files[1:]:
#     df = pd.concat([df, pd.read_csv(file)], axis = 0)
#     #df.to_csv('./csv/all_exercises.csv', index=False)

# df.sample(10)

In [ ]:
df.isna().sum()

In [ ]:
df[~df.year.isna()]

In [ ]:
df.shape

In [ ]:
df.subject.unique()

In [ ]:
df.exercise.unique()

In [ ]:
df.exercise = df.exercise.apply(lambda x: x.replace('Opcióon', 'Opción').replace('Opcion', 'Opción'))

In [ ]:
def add_comma(string):
    if len(string.split(' ')) == 4 and ',' not in string:
        words = string.split(' ')
        return f'{words[0]} {words[1]}, {words[2]} {words[3]}'
    else:
        return string

In [ ]:
df.exercise = df.exercise.apply(add_comma)

In [ ]:
df.to_csv(f'./csv/all_exercises_{df.subject.unique()[0]}.csv', index=False)